In [1]:
import pandas as pd
from tqdm import tqdm
from urllib.request import urlopen
from urllib.parse import quote
from rdkit import Chem
import numpy as np

from gensim.models import word2vec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [22]:
df = pd.read_csv("Task/train_md_count.csv")

In [23]:
dt = pd.read_csv("Task/train_from_rdkit.csv")

In [24]:
df = df.loc[:, "Smiles":]

In [25]:
dt

,Unnamed: 0,Smiles,Active,Aliphatic carboxylic acids,aliphatic hydroxyl groups,aliphatic hydroxyl groups excluding tert-OH,N functional groups attached to aromatics,Aromatic carboxylic acide,aromatic nitrogens,aromatic amines,...,value_HBA,value_HBD,value_PSA,value_ROTB,value_AROM,value_ALERTS,qed,weights_max,weights_mean,weights_none
0,0,COc1ccc2[nH]cc(CCN)c2c1,False,0,0,0,0,0,1,1,...,2,2,51.04,3,2,0,0.773221,0.765483,0.773221,0.820536
1,1,CCCN1CCC[C@H](c2cccc(O)c2)C1.Cl,False,0,0,0,0,0,0,0,...,2,1,23.47,3,1,0,0.894947,0.894548,0.894947,0.877186
2,2,O=C(NO)c1cnc(N2CCN(S(=O)(=O)c3ccc4ccccc4c3)CC2...,False,0,0,0,0,0,2,0,...,7,2,115.73,4,3,2,0.488593,0.465940,0.488593,0.491701
3,3,Nc1cccc(CNC(=O)c2ccc(Oc3ccc(OCc4cccc(F)c4)cc3)...,False,0,0,0,1,0,1,0,...,5,2,86.47,8,4,1,0.369974,0.419579,0.369974,0.396698
4,4,Fc1ccccc1CNCc1ccc(-c2ccnc3[nH]ccc23)cc1,False,0,0,0,0,0,2,1,...,2,2,40.71,5,4,0,0.558322,0.636613,0.558322,0.564299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5552,5552,O=C(Oc1ccc([N+](=O)[O-])cc1)N1CCC(C(O)(c2ccc3c...,False,0,1,0,0,0,0,0,...,9,1,129.83,5,3,2,0.388542,0.379894,0.388542,0.351634
5553,5553,Nc1nonc1/C(=N/O)Nc1ccc(F)c(Br)c1,False,0,0,0,1,0,2,0,...,6,3,109.56,2,2,4,0.337228,0.244981,0.337228,0.469045
5554,5554,Oc1cccc2cccnc12,False,0,0,0,0,0,1,0,...,2,1,33.12,0,2,0,0.614103,0.610171,0.614103,0.681858
5555,5555,OC(c1ccc(-c2ccc(CN3CCN(Cc4ccncc4)CC3)cc2)c(F)c...,False,0,1,0,0,0,1,0,...,4,1,39.60,6,3,1,0.429095,0.456986,0.429095,0.486160


In [26]:
dt = dt.loc[:, "Aliphatic carboxylic acids":]

In [27]:
data = pd.concat([df, dt], axis=1)

In [28]:
data

,Smiles,Active,BertzCT,GATSm3,Chiv4pc,SPP,CATS_DA4,CATS_DA1,Tpc,CATS_DP8,...,value_HBA,value_HBD,value_PSA,value_ROTB,value_AROM,value_ALERTS,qed,weights_max,weights_mean,weights_none
0,COc1ccc2[nH]cc(CCN)c2c1,False,2.637,0.845,0.742,0.663,0.000,0.000,1.289,0.0,...,2,2,51.04,3,2,0,0.773221,0.765483,0.773221,0.820536
1,CCCN1CCC[C@H](c2cccc(O)c2)C1.Cl,False,2.532,0.032,0.968,0.801,0.000,0.000,1.349,0.0,...,2,1,23.47,3,1,0,0.894947,0.894548,0.894947,0.877186
2,O=C(NO)c1cnc(N2CCN(S(=O)(=O)c3ccc4ccccc4c3)CC2...,False,3.058,1.248,3.031,0.615,0.182,0.091,2.573,0.0,...,7,2,115.73,4,3,2,0.488593,0.465940,0.488593,0.491701
3,Nc1cccc(CNC(=O)c2ccc(Oc3ccc(OCc4cccc(F)c4)cc3)...,False,3.089,1.197,1.115,0.742,0.200,0.000,2.863,0.0,...,5,2,86.47,8,4,1,0.369974,0.419579,0.369974,0.396698
4,Fc1ccccc1CNCc1ccc(-c2ccnc3[nH]ccc23)cc1,False,2.995,1.094,1.205,0.513,0.000,0.000,1.651,0.0,...,2,2,40.71,5,4,0,0.558322,0.636613,0.558322,0.564299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5552,O=C(Oc1ccc([N+](=O)[O-])cc1)N1CCC(C(O)(c2ccc3c...,False,3.123,1.061,2.725,0.869,0.000,0.000,3.905,0.0,...,9,1,129.83,5,3,2,0.388542,0.379894,0.388542,0.351634
5553,Nc1nonc1/C(=N/O)Nc1ccc(F)c(Br)c1,False,2.778,1.052,1.041,0.739,0.250,0.083,1.842,0.0,...,6,3,109.56,2,2,4,0.337228,0.244981,0.337228,0.469045
5554,Oc1cccc2cccnc12,False,2.578,0.956,0.428,0.799,0.000,0.000,0.981,0.0,...,2,1,33.12,0,2,0,0.614103,0.610171,0.614103,0.681858
5555,OC(c1ccc(-c2ccc(CN3CCN(Cc4ccncc4)CC3)cc2)c(F)c...,False,3.070,0.841,2.566,0.799,0.000,0.000,2.941,0.0,...,4,1,39.60,6,3,1,0.429095,0.456986,0.429095,0.486160


In [29]:
data.to_csv("Task/train_all.csv")

In [22]:
protein = 'CC[C@H](C)[C@H](NC(=O)CNC(=O)[C@H](C)NC(=O)[C@H](CO)NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC(C)C)NC(=O)[C@@H]1CCCN1C(=O)[C@H](C)NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC(C)C)NC(=O)[C@@H](NC(=O)[C@H](CCC(=O)O)NC(=O)[C@H](Cc1c[nH]cn1)NC(=O)[C@H](CCC(N)=O)NC(=O)[C@H](CC(N)=O)NC(=O)[C@H](Cc1ccccc1)NC(=O)[C@H](CC(N)=O)NC(=O)[C@H](CCC(=O)O)NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)[C@H](C)NC(=O)[C@H](CO)NC(=O)[C@H](CCCC[NH3+])NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CCC(N)=O)NC(=O)[C@@H](NC(=O)[C@H](CC(=O)O)NC(=O)[C@H](CCC(=O)O)NC(=O)CNC(=O)[C@H](CCCC[NH3+])NC(=O)[C@H](CC(N)=O)NC(=O)[C@@H](NC(=O)[C@H](CC(N)=O)NC(=O)[C@@H]1CCCN1C(=O)CNC(=O)[C@@H](NC(=O)[C@@H](NC(=O)[C@H](Cc1c[nH]cn1)NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CS)NC(=O)[C@H](Cc1c[nH]c[nH+]1)NC(=O)[C@H](CCCC[NH3+])NC(=O)[C@H](C)NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CC(N)=O)NC(=O)[C@H](Cc1c[nH]cn1)NC(=O)CNC(=O)[C@H](CO)NC(=O)[C@H](CC(C)C)NC(=O)[C@@H](NC(=O)[C@H](CS)NC(=O)[C@H](CO)NC(=O)CNC(=O)CNC(=O)[C@@H](NC(=O)[C@H](CCCC[NH3+])NC(=O)[C@H](CC(C)C)NC(=O)[C@@H]1CCCN1C(=O)CNC(=O)[C@H](CC(N)=O)NC(=O)[C@@H](NC(=O)[C@H](C)NC(=O)[C@@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)[C@H](CC(=O)O)NC(=O)[C@H](CC(=O)O)NC(=O)[C@H](CO)NC(=O)[C@H](CCC(=O)O)NC(=O)[C@@H](NC(=O)[C@H](CCC(N)=O)NC(=O)[C@H](CCSC)NC(=O)[C@H](C)NC(=O)[C@H](CC(N)=O)NC(=O)[C@H](CC(N)=O)NC(=O)[C@@H](NC(=O)[C@H](C)NC(=O)[C@H](CCCC[NH3+])NC(=O)[C@H](CC(N)=O)NC(=O)[C@H](CC(C)C)NC(=O)[C@H](C)NC(=O)CNC(=O)[C@H](C)NC(=O)[C@@H](NC(=O)CNC(=O)CNC(=O)CNC(=O)[C@H](Cc1c[nH]cn1)NC(=O)[C@H](CCCC[NH3+])NC(=O)[C@H](CC(C)C)NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)[C@@H](NC(=O)[C@H](CC(N)=O)NC(=O)[C@H](C)NC(=O)[C@H](C)NC(=O)[C@H](CC(N)=O)NC(=O)[C@@H](NC(=O)[C@@H](NC(=O)[C@@H](NC(=O)[C@@H](NC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCC[NH3+])NC(=O)[C@@H](NC(=O)[C@H](CCCC[NH3+])NC(=O)[C@H](CCCC[NH3+])NC(=O)[C@H](C)NC(=O)[C@H](CCC(=O)O)NC(=O)[C@H](CCC(=O)O)NC(=O)[C@@H](NC(=O)[C@@H](NC(=O)[C@H](CC(=O)O)NC(=O)[C@H](C)NC(=O)[C@H](CC(N)=O)NC(=O)[C@H](CCCC[NH3+])NC(=O)[C@@H](NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)[C@@H](NC(=O)[C@H](CC(N)=O)NC(=O)[C@H](CC(=O)O)NC(=O)[C@@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CCCC[NH3+])NC(=O)[C@H](CC(C)C)NC(=O)[C@H](Cc1ccc(O)cc1)NC(=O)CNC(=O)[C@H](CO)NC(=O)[C@H](Cc1ccccc1)NC(=O)[C@H](CO)NC(=O)[C@H](CC(N)=O)NC(=O)[C@@H](NC(=O)[C@@H](N)CCC(=O)O)C(C)C)[C@@H](C)O)C(C)C)[C@@H](C)CC)[C@@H](C)CC)C(C)C)C(C)C)[C@@H](C)O)C(C)C)C(C)C)C(C)C)C(C)C)C(C)C)[C@@H](C)O)C(C)C)[C@@H](C)CC)[C@@H](C)O)C(C)C)C(C)C)C(C)C)C(C)C)C(C)C)[C@@H](C)CC)C(C)C)C(=O)N[C@@H](Cc1ccccc1)C(=O)NCC(=O)N[C@@H](C)C(=O)N[C@@H](CC(=O)O)C(=O)N1CCC[C@H]1C(=O)N[C@H](C(=O)N[C@@H](Cc1c[nH]c[nH+]1)C(=O)N[C@@H](CO)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](CCCNC(N)=[NH2+])C(=O)N[C@H](C(=O)N[C@@H](CS)C(=O)N[C@H](C(=O)N[C@@H](CC(=O)O)C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@@H](CCCNC(N)=[NH2+])C(=O)N[C@H](C(=O)N[C@@H](CC(N)=O)C(=O)N[C@H](C(=O)N[C@@H](Cc1ccc(O)cc1)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](C)C(=O)N[C@H](C(=O)N[C@@H](Cc1ccccc1)C(=O)N[C@@H](CC(=O)O)C(=O)N[C@@H](CCCC[NH3+])C(=O)N[C@@H](CC(N)=O)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](Cc1ccc(O)cc1)C(=O)N[C@@H](CC(=O)O)C(=O)N[C@@H](CCCC[NH3+])C(=O)N[C@@H](CC(C)C)C(=O)N[C@H](C(=O)N[C@@H](CO)C(=O)N[C@@H](CO)C(=O)N[C@@H](Cc1ccccc1)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](CCC(=O)O)C(=O)O)C(C)C)C(C)C)C(C)C)[C@@H](C)O)C(C)C)[C@@H](C)O)C(C)C)C(C)C)[C@@H](C)CC.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O.O'